# Predicting the Age of Crabs: Enhancing Predictive Performance with Feature Engineering

In a previous notebook, we took a preliminary stab at predicting crab ages based on a set of basic biological and physical features. We used a dataset containing information on various aspects such as the crab's gender, body dimensions (length, diameter, height), weight (total, shucked, viscera, shell), and age. While we achieved a decent prediction accuracy with these basic features, there's potential for enhancement.

Previous Notebook - https://www.kaggle.com/code/pandeyg0811/mae-1-33-eda-ensemble

## Introduction

In this notebook, we're delving deeper into the world of crustacean life-cycles, specifically crabs. Crabs are fascinating creatures with diverse biological characteristics, playing crucial roles in marine ecosystems. A key attribute that's often challenging to estimate but crucial for biological and ecological studies is the age of a crab. Understanding the age distribution of a crab population can significantly contribute to population dynamics, growth rates, lifecycle understanding, and conservation strategies.

## Import Libraries

In [3]:
import gc
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
import xgboost as xgb
import optuna

import warnings
warnings.simplefilter("ignore")

C:\Users\Pranay\anaconda3\envs\checkenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing Dataset

In [4]:
dataset = 'C:/Users/Pranay/Desktop/Regression with a Crab Age Dataset/Dataset/'

df_train = pd.read_csv(dataset + "train.csv")
df_test = pd.read_csv(dataset + "test.csv")
df_original = pd.read_csv(dataset + "CrabAgePrediction.csv")

df_train["Data Type"] = 0
df_test["Data Type"] = 1
df_original["Data Type"] = 2


# label encoding (feature "Sex" is categorical)
le = LabelEncoder()
df_train["Sex"] = le.fit_transform(df_train["Sex"])
df_test["Sex"] = le.transform(df_test["Sex"])
df_original["Sex"] = le.transform(df_original["Sex"])

# concatenate datasets
df_concat = pd.concat([df_train.drop('id',axis=1), df_original], ignore_index=True)
df_concat = df_concat.drop_duplicates()
df_all = pd.concat([df_concat, df_test.drop('id',axis=1)], ignore_index=True)
df_all

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Data Type
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9.0,0
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8.0,0
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9.0,0
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11.0,0
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8.0,0
...,...,...,...,...,...,...,...,...,...,...
127307,0,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309,NaN,1
127308,1,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687,NaN,1
127309,0,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345,NaN,1
127310,0,1.2375,0.9500,0.2875,15.663099,6.095142,3.727959,4.961163,NaN,1


## Imputing Height variable

In [5]:
# repalce some wrong data (Height=0) with random forest prediction
h1 = df_all[df_all["Height"] != 0]
h0 = df_all[df_all["Height"] == 0]
print(h1.shape, h0.shape)

x_h1 = h1.drop(columns=[ "Height", "Age", "Data Type"], axis=1)
y_h1 = h1["Height"]
x_h0 = h0.drop(columns=[ "Height", "Age", "Data Type"], axis=1)

rfr = RandomForestRegressor(n_jobs=-1, random_state=28)
rfr.fit(x_h1, y_h1)
preds_height = rfr.predict(x_h0)

cnt = 0
for i in range(len(df_all)):
    if df_all.loc[i, "Height"] == 0:
        df_all.loc[i, "Height"] = preds_height[cnt]
        cnt += 1

df_all["Height"].describe()

(127272, 10) (40, 10)


count    127312.000000
mean          0.348414
std           0.092400
min           0.012500
25%           0.300000
50%           0.362500
75%           0.412500
max           2.825000
Name: Height, dtype: float64

## Modelling

In [6]:
lgb_params = {
    "objective": "regression_l1", # ="mae"
    "metric": "mae",
    "learning_rate": 0.03, # 0.1
    "n_estimators": 10000,
    "max_depth": 8, # -1, 1-16(3-8)
    "num_leaves": 255, # 31, 2-2**max_depth
    "feature_fraction": 0.4, # 1.0, 0.1-1.0, 0.4
    "min_data_in_leaf": 256, # 20, 0-300
    "subsample": 0.4, # 1.0, 0.01-1.0
    "reg_alpha": 0.1, # 0.0, 0.0-10.0, 0.1
    "reg_lambda": 0.1, # 0.0, 0.0-10.0, 0.1
    ###"subsample_freq": 0, # 0-10
    ###"max_bin": 255, # 32-512
    ###"min_gain_to_split": 0.0, # 0.0-15.0
    ###"subsample_for_bin": 200000, # 30-len(x_train)
    ###"boosting": "dart", # "gbdt"
    ###"device_type": "gpu", # "cpu"
}

### Train and Test Data Split

In [7]:
train = df_all[df_all['Data Type'] != 1]
train.reset_index(drop=True, inplace=True)

y_train = train["Age"].astype(int)
x_train = train.drop(columns=["Age", "Data Type"], axis=1)

x_test = df_all[df_all["Data Type"] == 1]
x_test.reset_index(drop=True, inplace=True)
x_test.drop(columns=["Age", "Data Type"], inplace=True)

### LightGBM Model

In [8]:
def LightGBM(X, y, test_data, params):
    kf = list(KFold(n_splits=10, shuffle=True, random_state=100).split(X, y))
    preds, models = [], []
    oof = np.zeros(len(X))
    imp = pd.DataFrame()
    
    for nfold in np.arange(10):
        print("-"*30, "fold:", nfold, "-"*30)
        
        # set train/valid data
        idx_tr, idx_va = kf[nfold][0], kf[nfold][1]
        x_tr, y_tr = X.loc[idx_tr, :], y.loc[idx_tr]
        x_va, y_va = X.loc[idx_va, :], y.loc[idx_va]
        
        # training
        model = lgb.LGBMRegressor(**params)
        model.fit(x_tr, y_tr,
                eval_set=[(x_tr, y_tr), (x_va, y_va)],
                early_stopping_rounds=300,
                verbose=500,
        )
        models.append(model)
        
        # validation
        pred_va = model.predict(x_va)
        oof[idx_va] = pred_va
        print("MAE(valid)", nfold, ":", "{:.4f}".format(mean_absolute_error(y_va, pred_va)))
        
        # prediction
        pred_test = model.predict(test_data)
        preds.append(pred_test)
        
        # importance
        _imp = pd.DataFrame({"features": X.columns, "importance": model.feature_importances_, "nfold": nfold})
        imp = pd.concat([imp, _imp], axis=0, ignore_index=True)
    
    imp = imp.groupby("features")["importance"].agg(["mean", "std"])
    imp.columns = ["importance", "importance_std"]
    imp["importance_cov"] = imp["importance_std"] / imp["importance"]
    imp = imp.reset_index(drop=False)
    display(imp.sort_values("importance", ascending=False, ignore_index=True))
    
    return preds, models, oof, imp

## Model without Feature Engineering

In [9]:
# Training
preds_lgb, models_lgb, oof_lgb, imp_lgb = LightGBM(x_train, y_train, x_test, lgb_params)

# MAE for LightGBM
oof_lgb_round = np.zeros(len(oof_lgb), dtype=int)
for i in range(len(oof_lgb)):
    oof_lgb_round[i] = int((oof_lgb[i] * 2 + 1) // 2)

print("MAE(int):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb_round)))
print("MAE(float):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb)))

# visualization of predictions by test-data
mean_preds_lgb = np.mean(preds_lgb, axis=0)
mean_preds_lgb_round = np.zeros(len(mean_preds_lgb), dtype=int)
for i in range(len(mean_preds_lgb_round)):
    mean_preds_lgb_round[i] = int((mean_preds_lgb[i] * 2 + 1) // 2)


------------------------------ fold: 0 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256
[500]	training's l1: 1.33428	valid_1's l1: 1.36354
[1000]	training's l1: 1.31994	valid_1's l1: 1.35797
[1500]	training's l1: 1.31145	valid_1's l1: 1.35547
[2000]	training's l1: 1.30523	valid_1's l1: 1.35485
[2500]	training's l1: 1.30059	valid_1's l1: 1.35448
[3000]	training's l1: 1.29627	valid_1's l1: 1.35417
MAE(valid) 0 : 1.3541
------------------------------ fold: 1 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256
[500]	training's l1: 1.3256	val

,features,importance,importance_std,importance_cov
0,Shell Weight,15735.9,1599.736746,0.101662
1,Weight,13990.8,1461.621991,0.104470
2,Shucked Weight,12033.5,1286.067067,0.106874
3,Viscera Weight,11847.7,1387.583915,0.117118
4,Length,10217.5,1114.242668,0.109052
5,Diameter,9275.6,1016.140432,0.109550
6,Height,7342.8,1121.561590,0.152743
7,Sex,2568.3,293.875654,0.114424


MAE(int): 1.3423
MAE(float): 1.3620


# Reducing Error with with Feature Engineering

Now, we are ready to take our analysis a step further. In this notebook, we're going to incorporate feature engineering - a powerful tool that allows us to create new features from existing ones, thereby injecting our domain knowledge into the models, enhancing the richness of our data, and potentially boosting the predictive performance of our models.

Feature engineering is often what separates a good model from a great one. It can help us uncover complex patterns in the data that basic models might overlook. Given the intricacy of biological entities like crabs, there's a great deal of potential in engineering new features that better capture the nuanced aspects of a crab's biology and life history.

We'll be exploring a range of feature engineering techniques such as creating ratio features to capture relative size differences, geometric features to encapsulate physical properties, polynomial features to capture non-linear relationships, logarithmic transformations to manage extreme values, binning to simplify relationships with the target variable, and creating new weight-related features to provide a deeper look into weight distribution.

By the end of this notebook, we will have a much-improved model for predicting crab age, setting the stage for more accurate and informed biological and ecological studies. Let's dive in!


### 1. Ratio-based features

- **Viscera Ratio**: The proportion of the crab's weight that comes from the viscera. This might be useful in understanding how the internal organ development correlates with the crab's age. 
   * Formula: `Viscera Ratio = Viscera Weight / Weight`

### 2. Geometric features

- **Surface Area**: Surface area of the crab computed as if it were a box. This feature could encapsulate the crab's overall size in a different way that the individual dimensions do not capture.
   * Formula: `Surface Area = 2 * (Length * Diameter + Length * Height + Diameter * Height)`
- **Volume**: Volume of the crab computed as if it were a box. Similar to the surface area, this provides a holistic measure of the crab's size.
   * Formula: `Volume = Length * Diameter * Height`
- **Density**: Density of the crab computed based on its weight and volume. It might help understand if older crabs tend to be denser or lighter for their size.
   * Formula: `Density = Weight / Volume`

### 3. Weight-related features

- **Shell-to-Body Ratio**: Ratio of the shell weight to the sum of total weight and shell weight. This can help understand if the shell development has a correlation with the crab's age.
   * Formula: `Shell-to-Body Ratio = Shell Weight / (Weight + Shell Weight)`
- **Meat Yield**: Ratio of the shucked weight to the sum of total weight and shell weight. It may capture if older crabs tend to have more or less meat relative to their total size.
   * Formula: `Meat Yield = Shucked Weight / (Weight + Shell Weight)`
- **Weight_wo_Viscera**: Weight of the crab without the viscera. It can help to examine how much of the crab's weight comes from non-viscera parts and if that changes with age.
   * Formula: `Weight_wo_Viscera = Shucked Weight - Viscera Weight`

### 4. Polynomial features

- **Length^2**: Squared length of the crab. It might capture any non-linear relationships between length and age.
   * Formula: `Length^2 = Length ** 2`
- **Diameter^2**: Squared diameter of the crab. Similar to length squared, this might capture any non-linear relationships between diameter and age.
   * Formula: `Diameter^2 = Diameter ** 2`

### 5. Logarithmic transformations

- **Log Weight**: Logarithm of the crab's weight. This might help deal with any extreme or skewed weight values and can sometimes help linearize relationships with the target variable.
   * Formula: `Log Weight = log(Weight + 1)`

### 6. Binning

- **Length Bins**: Discretization of the length into 4 bins. This simplifies the relationship of length with age and helps deal with any irregularities or noise in the relationship.
   * Formula: `Length Bins = pd.cut(Length, bins=4, labels=False)`

## Feature Engineering

In [10]:
df_all["Viscera Ratio"] = df_all["Viscera Weight"] / df_all["Weight"]
df_all["Shell Ratio"] = df_all["Shell Weight"] / df_all["Weight"]
df_all["Surface Area"] = 2 * (df_all["Length"] * df_all["Diameter"] + df_all["Length"] * df_all["Height"] + df_all["Diameter"] * df_all["Height"])
df_all["Volume"] = df_all["Length"] * df_all["Diameter"] * df_all["Height"]
df_all["Density"] = df_all["Weight"] / df_all["Volume"]
df_all['Shell-to-Body Ratio'] = df_all['Shell Weight'] / (df_all['Weight'] + df_all['Shell Weight'])
df_all['Meat Yield'] = df_all['Shucked Weight'] / (df_all['Weight'] + df_all['Shell Weight'])
df_all['Body Condition Index'] = np.sqrt(df_all['Length'] * df_all['Weight'] * df_all['Shucked Weight'])
df_all['Pseudo BMI']=df_all['Weight']/(df_all['Height']**2)
df_all['Len-to-Diam']=df_all['Length']/df_all['Diameter']
df_all['wieght-to-Viswieght']=df_all['Weight']/df_all['Viscera Weight']
df_all['wieght-to-Shellwieght']=df_all['Weight']/df_all['Shell Weight']
df_all['wieght-to-Shckwieght']=df_all['Weight']/df_all['Shucked Weight']
df_all["Weight_wo_Viscera"] = df_all['Shucked Weight'] - df_all['Viscera Weight']
df_all['Length^2'] = df_all['Length'] ** 2
df_all['Diameter^2'] = df_all['Diameter'] ** 2
df_all['Log Weight'] = np.log(df_all['Weight'] + 1) 
df_all['Length Bins'] = pd.cut(df_all['Length'], bins=4, labels=False)

In [11]:
train = df_all[df_all['Data Type'] != 1]
train.reset_index(drop=True, inplace=True)

y_train = train["Age"].astype(int)
x_train = train.drop(columns=["Age", "Data Type"], axis=1)

x_test = df_all[df_all["Data Type"] == 1]
x_test.reset_index(drop=True, inplace=True)
x_test.drop(columns=["Age", "Data Type"], inplace=True)

## Model with Feature Engineering

In [12]:
# Training
preds_lgb_fe, models_lgb_fe, oof_lgb_fe, imp_lgb_fe = LightGBM(x_train, y_train, x_test, lgb_params)

# MAE for LightGBM
oof_lgb_round = np.zeros(len(oof_lgb), dtype=int)
for i in range(len(oof_lgb)):
    oof_lgb_round[i] = int((oof_lgb[i] * 2 + 1) // 2)

print("MAE(int):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb_round)))
print("MAE(float):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb)))

# visualization of predictions by test-data
mean_preds_lgb = np.mean(preds_lgb, axis=0)
mean_preds_lgb_round = np.zeros(len(mean_preds_lgb), dtype=int)
for i in range(len(mean_preds_lgb_round)):
    mean_preds_lgb_round[i] = int((mean_preds_lgb[i] * 2 + 1) // 2)


------------------------------ fold: 0 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256
[500]	training's l1: 1.30562	valid_1's l1: 1.35609
[1000]	training's l1: 1.2915	valid_1's l1: 1.3548
MAE(valid) 0 : 1.3545
------------------------------ fold: 1 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256
[500]	training's l1: 1.30326	valid_1's l1: 1.39262
[1000]	training's l1: 1.28514	valid_1's l1: 1.39071
[1500]	training's l1: 1.2721	valid_1's l1: 1.38997
[2000]	training's l1: 1.26157	valid_1's l1: 1.38987
MAE(valid) 1 : 1.3898
-------------

,features,importance,importance_std,importance_cov
0,Shell Weight,2965.4,618.881823,0.208701
1,Meat Yield,2780.3,697.401853,0.250837
2,Shell Ratio,2384.9,557.403195,0.233722
3,wieght-to-Shckwieght,2263.1,591.643183,0.261430
4,Density,2236.6,672.038061,0.300473
5,Pseudo BMI,2139.2,654.430507,0.305923
6,Shucked Weight,2129.4,519.336585,0.243889
7,Viscera Ratio,2101.5,533.271090,0.253757
8,Len-to-Diam,2068.8,664.245905,0.321078
9,Weight,1992.3,377.085622,0.189272


MAE(int): 1.3423
MAE(float): 1.3620


***

<BR>
    
    
<div style="text-align: center;">
   <span style="font-size: 4.5em; font-weight: bold; font-family: Arial;">THANK YOU!</span>
</div>

<div style="text-align: center;">
    <span style="font-size: 5em;">✔️</span>
</div>

<br>

<div style="text-align: center;">
   <span style="font-size: 1.4em; font-weight: bold; font-family: Arial; max-width:1200px; display: inline-block;">
       These features helped me achieved reduced error rate and better leaderboard ranking. If you also found them helpful, please provide an upvote!

   </span>
</div>

<br>

<br>

<div style="text-align: center;">
   <span style="font-size: 1.2em; font-weight: bold;font-family: Arial;">@Gaurav Pandey</span>
</div>